# Analyzing Meridians Artblocks

## Get the meridians

In [1]:
import requests
from helpers import *

OPENSEA_URL = "https://api.opensea.io/api/v1/assets"
MERIDIAN_IDS = list(range(163000000, 163001000))


def get_meridians():
    querystring = {
        "asset_contract_address": "0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270",
    }
    assets = []
    for token_ids in chunks(MERIDIAN_IDS, 20):
        print(".", end="")
        querystring["token_ids"] = token_ids
        response = requests.request("GET", OPENSEA_URL, params=querystring)
        response.raise_for_status()
        assets.extend(response.json()["assets"])
    print(f" -- All {len(assets)} assets retrieved.")
    return assets

# FIXME Turn into a generator?

meridians = get_meridians()

.................................................. -- All 1000 assets retrieved.


## Build a dataframe

In [3]:
import pandas as pd
import numpy as np
from functools import reduce
import operator

_tt = []
for m in meridians:
    prob_score = (
        reduce(operator.mul, [t["trait_count"] / 1000 for t in m["traits"]], 1) * 1000
    )
    price = np.nan
    if m["sell_orders"] is not None:
        so = m["sell_orders"][0]
        price = float(so["current_price"]) / 10 ** int(
            so["payment_token_contract"]["decimals"]
        )
        # FIXME Also get the currency's name
    last_price = np.nan
    if m["last_sale"] is not None:
        ls = m["last_sale"]
        last_price = float(ls["total_price"]) / 10 ** int(
            ls["payment_token"]["decimals"]
        )

    row = [
        m["name"].split("#")[1],
        m["name"],
        price,
        last_price,
        prob_score,
    ]
    row.extend(
        t["value"].split(":")[1]
        for t in sorted(m["traits"], key=lambda x: x["value"])
        if ":" in t["value"]
    )
    row.append(m["permalink"])
    _tt.append(row)

df = pd.DataFrame(_tt)
df.columns = [
    "id",
    "name",
    "price",
    "lastprice",
    "prob_score",
    "chromaticity",
    "frequency",
    "palette",
    "structure",
    "style",
    "zoom",
    "link",
]
df = df.set_index("id").sort_values("lastprice", ascending=False)

In [4]:
df

,name,price,lastprice,prob_score,chromaticity,frequency,palette,structure,style,zoom,link
id,,,,,,,,,,,
738,Meridian #738,NaN,200.0,0.042095,Monochrome,Low,Other,Uniform,Charcoal,Far,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...
287,Meridian #287,NaN,200.0,0.002504,Monochrome,Standard,Gulf,Uniform,Prime,Medium,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...
70,Meridian #70,NaN,175.0,0.086479,Monochrome,Standard,Other,Uniform,Charcoal,Far,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...
624,Meridian #624,NaN,145.0,0.002497,Monochrome,Standard,Esker,Uniform,Prime,Far,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...
809,Meridian #809,NaN,80.0,0.037515,Monochrome,Gradual,Other,Uniform,Charcoal,Far,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...
...,...,...,...,...,...,...,...,...,...,...,...
996,Meridian #996,NaN,NaN,0.130420,Trichrome,Standard,Other,Variable,Polarized,Medium,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...
990,Meridian #990,NaN,NaN,2.536809,Polychrome,Gradual,Other,Uniform,Prismatic,Far,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...
988,Meridian #988,NaN,NaN,0.136793,Trichrome,Standard,Other,Uniform,Topographic,Far,https://opensea.io/assets/0xa7d8d9ef8d8ce8992d...


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (20, 10)
pd.set_option("display.max_columns", None)
pd.plotting.register_matplotlib_converters()

sns.boxplot(x=df["style"], y=df.lastprice, hue=df["chromaticity"])
df.groupby("style").nunique()
df[df.name == "Meridian #666"]